In [5]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# Define the correct column names
column_names = ['ET', 'V', 'N', 'SA', 'IA', 'RL', 'RE', 'P', 'FX', 'FY', 'FZ', 'MX', 'MZ',
                'NFX', 'NFY', 'RST', 'TSTI', 'TSTC', 'TSTO', 'AMBTMP', 'SR']

# Define the Pacejka model function
def pacejka_model(alpha, B, C, D, E, F):
    return -(D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F))

# Function to calculate the local slope using the difference method
def calculate_local_slope_pacejka(x_values, y_values):
    slopes = np.diff(y_values) / np.diff(x_values)
    return np.append(slopes, np.nan)

# Function to load and prepare the dataframe
def load_and_prepare_dataframe(file_path):
    try:
        df = pd.read_csv(file_path, delim_whitespace=True, names=column_names, on_bad_lines='skip')
        print(f"Successfully loaded {file_path}")
    except Exception as e:
        print(f"Failed to load {file_path}. Error: {e}")
        return None

    df = df.iloc[2:].reset_index(drop=True)
    if len(df) > 0:
        return df
    else:
        print(f"File {file_path} has zero length after preprocessing.")
        return None

# Function to find the shifting factors between camber angles
def find_shifting_factors(df, camber_angle1, camber_angle2):
    df1 = df[df['IA'] == camber_angle1]
    df2 = df[df['IA'] == camber_angle2]
    
    if df1.empty or df2.empty:
        print(f"No data available for camber angles {camber_angle1} or {camber_angle2}.")
        return None, None

    # Fit the Pacejka model to both sets of data
    x1, y1 = df1['SA'], df1['FY']
    x2, y2 = df2['SA'], df2['FY']

    try:
        popt1, _ = curve_fit(pacejka_model, x1, y1, p0=[0.5, 1.2, max(y1), 1, 0], maxfev=175000)
        popt2, _ = curve_fit(pacejka_model, x2, y2, p0=[0.5, 1.2, max(y2), 1, 0], maxfev=175000)
        
        # Generate fitted curves
        fitted_curve1 = pacejka_model(x1, *popt1)
        fitted_curve2 = pacejka_model(x2, *popt2)
        
        # Calculate shifting factors (average shift)
        shift_fy = np.mean(fitted_curve2 - fitted_curve1)
        shift_sa = np.mean(x2 - x1)
        
        return shift_fy, shift_sa

    except Exception as e:
        print(f"Error fitting curves for shifting factors: {e}")
        return None, None

# Adjusted function to process data and find shifting factors around -1° and -0.5°
def process_and_save_plot_with_shifting(file_paths, output_directory):
    # Define small ranges around -1° and -0.5° camber angles for approximation
    camber_range_1 = (-1.1, -0.9)
    camber_range_2 = (-0.6, -0.4)
    
    for file_path in file_paths:
        # Load and prepare the dataframe
        df = load_and_prepare_dataframe(file_path)
        if df is None:
            continue

        print(f"Processing file: {file_path}")

        # Convert columns to numeric and clean data
        df['FY'] = pd.to_numeric(df['FY'], errors='coerce') * -1 / 1000  # Convert FY to kN
        df['SA'] = pd.to_numeric(df['SA'], errors='coerce')
        df['IA'] = pd.to_numeric(df['IA'], errors='coerce')  # Convert camber angle to numeric
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(subset=['FY', 'SA', 'IA'], inplace=True)

        # Filter data around -1° and -0.5° camber angles
        df_around_1 = df[(df['IA'] >= camber_range_1[0]) & (df['IA'] <= camber_range_1[1])]
        df_around_05 = df[(df['IA'] >= camber_range_2[0]) & (df['IA'] <= camber_range_2[1])]

        if df_around_1.empty or df_around_05.empty:
            print(f"No sufficient data in the desired camber ranges for file: {file_path}")
            continue

        # Fit the Pacejka model to the data around -1° and -0.5° camber angles
        try:
            # Around -1° camber
            x_data_1 = df_around_1['SA']
            y_data_1 = df_around_1['FY']
            popt_1, _ = curve_fit(pacejka_model, x_data_1, y_data_1, p0=[0.5, 1.2, max(y_data_1), 1, 0], maxfev=175000)
            fitted_curve_1 = pacejka_model(x_data_1, *popt_1)
            
            # Around -0.5° camber
            x_data_05 = df_around_05['SA']
            y_data_05 = df_around_05['FY']
            popt_05, _ = curve_fit(pacejka_model, x_data_05, y_data_05, p0=[0.5, 1.2, max(y_data_05), 1, 0], maxfev=175000)
            fitted_curve_05 = pacejka_model(x_data_05, *popt_05)
        except Exception as e:
            print(f"Error fitting curves for {file_path}: {e}")
            continue

        # Calculate shifting factors
        # Here we assume a linear shift by calculating the difference between the means of the fitted curves
        lateral_shift = np.mean(fitted_curve_05) - np.mean(fitted_curve_1)
        print(f"Lateral shift factor between -1° and -0.5° camber for {file_path}: {lateral_shift:.4f}")

In [6]:
# Example usage on a list of file paths
file_paths = [
    '/Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat',
    '/Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw6.dat',
]

# Specify the output directory for saving the HTML files
output_directory = "/Users/peytonboone/Desktop/feb/fall2024/findingShiftingFactor"

# Call the function to process and save 3D graphs as HTML files using Plotly
process_and_save_plot_with_shifting(file_paths, output_directory)

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_72127/3933840260.py:23: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_72127/3933840260.py:23: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Successfully loaded /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat
Processing file: /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat
Error fitting curves for /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw4.dat: Optimal parameters not found: Number of calls to function has reached maxfev = 175000.


/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_72127/3933840260.py:23: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead

/var/folders/2n/12xxtc511cbd3qt_qc5sy6hm0000gn/T/ipykernel_72127/3933840260.py:23: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Successfully loaded /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw6.dat
Processing file: /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw6.dat
Lateral shift factor between -1° and -0.5° camber for /Users/peytonboone/Desktop/feb/fall2024/rawdata/B2356raw6.dat: -0.0468
